# **Ejercicio 1 (2 puntos)**


## Autores:

### José María García Ortiz & Francisco José Herrera Zapata.

<br>

<div style="background-color: #54c7ec; color: #fff; font-weight: 700; padding-left: 10px; padding-top: 5px; padding-bottom: 5px"><strong>ENUNCIADO:</strong></div>

Explica en detalle cómo funciona el modelo de Bahdanau, sobre la implementación que se ha usado en esta práctica y usando como apoyo el paper que se ha referenciado. Si se considera conveniente, usa el propio ejecuciones parciales (i.e. fragmentos) del propio código para proporcionar dichas explicaciones. Referencia también el mecanismo de visualización que se ha usado para los cuatro ejemplos que acabas de ver basados en las matrices de atención.

## **Preámbulo**

### Importación de librerías

In [ ]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import numpy as np
from torch.utils.data import TensorDataset, DataLoader, RandomSampler

import time
import math

import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
print(device)
print(torch.__version__)

cuda
2.5.1+cu121


In [7]:
print(torch.version.cuda)             # CUDA con la que fue compilado PyTorch
print(torch.cuda.is_available())      # Si la GPU está disponible
print(torch.cuda.get_device_name(0))  # Nombre de la GPU (si está disponible)

12.1
True
NVIDIA GeForce RTX 4060


### Fragmentos de código para emplear como base

#### Modelización de los diccionarios como clase

In [ ]:
SOS_token = 0
EOS_token = 1

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}     # Ponemos indice 0 a "SOS" e indice 1 a "EOS" en el diccionario de palabras por defecto
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [ ]:
# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427

def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )


def normalizeString(s):
    ''' Lowercase, trim, and remove non-letter characters '''

    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z!?]+", r" ", s)
    return s.strip()

In [ ]:
def readLangs(lang1, lang2, reverse=False):
    
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('../data/%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [ ]:
MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)

def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [ ]:
def prepareData(lang1, lang2, reverse=False):
    ''' Esta función encapsula todos los procesos de lectura y filtrado de pares de frases y creación de diccionarios en
    los respectivos idiomas que acabamos de ver '''

    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs

input_lang, output_lang, pairs = prepareData('eng', 'fra', True)
print(random.choice(pairs))

#### Encoder y decoder estándar (sin atención)

In [ ]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, dropout_p=0.1):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, input):
        embedded = self.dropout(self.embedding(input))
        output, hidden = self.gru(embedded)
        return output, hidden

In [ ]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)

    def forward(self, encoder_outputs, encoder_hidden, target_tensor=None):
        batch_size = encoder_outputs.size(0)
        decoder_input = torch.empty(batch_size, 1, dtype=torch.long, device=device).fill_(SOS_token)
        decoder_hidden = encoder_hidden
        decoder_outputs = []

        for i in range(MAX_LENGTH):
            decoder_output, decoder_hidden  = self.forward_step(decoder_input, decoder_hidden)
            decoder_outputs.append(decoder_output)

            if target_tensor is not None:
                # Teacher forcing: Feed the target as the next input
                decoder_input = target_tensor[:, i].unsqueeze(1) # Teacher forcing
            else:
                # Without teacher forcing: use its own predictions as the next input
                _, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze(-1).detach()  # detach from history as input

        decoder_outputs = torch.cat(decoder_outputs, dim=1)
        decoder_outputs = F.log_softmax(decoder_outputs, dim=-1)
        return decoder_outputs, decoder_hidden, None # We return `None` for consistency in the training loop

    def forward_step(self, input, hidden):
        output = self.embedding(input)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.out(output)
        return output, hidden

## **Resolución del ejercicio**

Antes de comenzar con los comentarios sobre el código, hemos creído conveniente dejar un breve esquema de notación basado en el [paper](https://arxiv.org/pdf/1409.0473.pdf) referenciado, con el que referirnos a los distintos elementos que componen la estructura encoder-decoder basada en atención representada por las clases ``BahdanauAttention`` y ``AttnDecoderRNN``.

**NOTACION:**

- $x:= (x_1, \ldots , x_{T_x}) \hspace{0.1 cm} \equiv \hspace{0.1 cm} \text{Frase de input al encoder}$

- $y:= (x_1, \ldots , y_{T_y}) \hspace{0.1 cm} \equiv \hspace{0.1 cm} \text{Frase de output del decoder}$

- $h_t:= f(x_t, h_{t-1}) \hspace{0.1 cm} \equiv \hspace{0.1 cm} \text{Estado oculto del encoder asociado al token } x_t \text{ del input } x$ (con $f$ una función no-lineal por determinar) $\hspace{0.1 cm} \equiv \hspace{0.1 cm}$ *anotaciones*

- $s_t= f(s_{i-1}, y_{i-1},c_i) \hspace{0.1 cm} \equiv \hspace{0.1 cm} \text{Estado oculto del decoder asociado al token } y_{t-1} \text{ del output } y$ 

- $c_i:= \sum_{j=1}^{T_x}\alpha_{ij}h_j \hspace{0.1 cm} \equiv \hspace{0.1 cm} \text{Vector de contexto dinámico en la } i$-ésima posición (como suma ponderada de las anotaciones)

- $\alpha_{ij}:= \frac{exp(e_{ij})}{\sum_{k=1}^{T_x}exp(e_{ik})} \hspace{0.1 cm} \equiv \hspace{0.1 cm} \text{Pesos de las anotaciones}$

- $e_{ij}:= a(s_{i-1}, h_j)$ $\hspace{0.1 cm}$ donde $a$ es un *método de alineación* con estructura de red neuronal feedforwad que se aprende junto con el resto de parámetros del modelo
 
<br>

Procedemos a analizar en detalle el código que implementa el modelo de atención de Bahdanau

In [ ]:
class BahdanauAttention(nn.Module):
    def __init__(self, hidden_size):
        super(BahdanauAttention, self).__init__()
        self.Wa = nn.Linear(hidden_size, hidden_size)
        self.Ua = nn.Linear(hidden_size, hidden_size)
        self.Va = nn.Linear(hidden_size, 1)

    def forward(self, query, keys):
        scores = self.Va(torch.tanh(self.Wa(query) + self.Ua(keys)))
        scores = scores.squeeze(2).unsqueeze(1)

        weights = F.softmax(scores, dim=-1)
        context = torch.bmm(weights, keys)  

        return context, weights

La clase ``BahdanauAtention`` es una subclase de ``nn.Module`` que modeliza una cabeza de atención del decoder de Bahdanau.

En el constructor solo se espera la variable ``hidden size`` que determina el tamaño de las matrices ``Wa`` y ``Ua``, encargadas respectivamente de ponderar un estado oculto del decoder (``query``) y de ponderar un estado oculto del encoder (``keys``). Por su parte, la matriz ``Va`` se encarga de parametrizar la salida del decoder, que es una proyección a un escalar que *mide* la atención de ``query`` respecto de ``keys`` (ambos tensores de pytorch).

Estas tres matrices constituyen los atributos de la clase.

Dentro de la clase solo encontramos además el método ``forward`` que se encarga de calcular unos scores de atención
$$\text{scores} = V_a \cdot tanh(W_a \cdot \text{query} \hspace{0.1 cm} + \hspace{0.1 cm} U_a \cdot \text{keys})$$
y después normalizarlos (línea 12) para obtener el vector de contexto (línea 13).

<br>

Tratando de esclarecer el *match* entre el código que calcula la atención y la notación de las ecuaciones propuestas en [la publicación](https://arxiv.org/pdf/1409.0473.pdf): 

$$
c_i = \sum_{j=1}^{T_x}\alpha_{ij}h_j \hspace{0.5 cm} , \hspace{0.5 cm} \alpha_{ij} = \frac{exp(e_{ij})}{\sum_{k=1}^{T_x}exp(e_{ik})} \hspace{0.5 cm} , \hspace{0.5 cm} e_{ij} = a(s_{i-1},h_j)
$$

Lo que tenemos en código es precisamente una particularización de $a$ y un cálculo alternativo de los pesos directamente como vector:

$$c_i = \sum_{j=1}^{T_x}\alpha_{ij}h_j \hspace{0.3 cm} \text{(línea 13)} \hspace{0.6 cm} , \hspace{0.5 cm}

\text{weights} \hspace{0.1 cm} \equiv \hspace{0.1 cm} \alpha_i :=  \text{softmax}(\text{scores}) \hspace{0.1 cm} \equiv \hspace{0.1 cm} (\alpha_{ij})_{j \in \{1,\ldots,T_x\}} =    \hspace{0.5 cm} , \hspace{0.5 cm}

a(\text{query},\text{key}) \hspace{0.1 cm} \equiv \hspace{0.1 cm} a(s_{i-1},h_j):= V_a \cdot tanh(W_a \cdot \text{query}(i) \hspace{0.1 cm} + \hspace{0.1 cm} U_a \cdot \text{key})$$

y cabe destacar que el cómputo de los alineamientos $a(\text{query},\text{key})$ se relizan en bloque (línea 9) para todo el conjunto de claves
$$\text{keys} = \{h_j : 1 \leq j  \leq T_x\}.$$

In [ ]:
class AttnDecoderRNN(nn.Module):

    def __init__(self, hidden_size, output_size, dropout_p=0.1):
        super(AttnDecoderRNN, self).__init__()
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.attention = BahdanauAttention(hidden_size)
        self.gru = nn.GRU(2 * hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(dropout_p)


    def forward(self, encoder_outputs, encoder_hidden, target_tensor=None):
        batch_size = encoder_outputs.size(0)
        decoder_input = torch.empty(batch_size, 1, dtype=torch.long, device=device).fill_(SOS_token)
        decoder_hidden = encoder_hidden
        decoder_outputs = []
        attentions = []

        for i in range(MAX_LENGTH):
            decoder_output, decoder_hidden, attn_weights = self.forward_step(
                decoder_input, decoder_hidden, encoder_outputs
            )
            decoder_outputs.append(decoder_output)
            attentions.append(attn_weights)

            if target_tensor is not None:
                # Teacher forcing: Feed the target as the next input
                decoder_input = target_tensor[:, i].unsqueeze(1) # Teacher forcing
            else:
                # Without teacher forcing: use its own predictions as the next input
                _, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze(-1).detach()  # detach from history as input

        decoder_outputs = torch.cat(decoder_outputs, dim=1)
        decoder_outputs = F.log_softmax(decoder_outputs, dim=-1)
        attentions = torch.cat(attentions, dim=1)

        return decoder_outputs, decoder_hidden, attentions


    def forward_step(self, input, hidden, encoder_outputs):
        embedded =  self.dropout(self.embedding(input))

        query = hidden.permute(1, 0, 2)
        context, attn_weights = self.attention(query, encoder_outputs)
        input_gru = torch.cat((embedded, context), dim=2)

        output, hidden = self.gru(input_gru, hidden)
        output = self.out(output)

        return output, hidden, attn_weights

En la clase ``AttnDecoderRNN`` se incluyen los atributos y métodos necesarios para programar el decoder basado en atención de Bahdanau.

En el constructor ``__init__`` de la clase se definen 5 atributos (``embedding``, ``attention``, ``gru``, ``out`` y ``dropout``), cada uno de los cuales representa una de las capas que componen el decoder. En primer lugar (línea 6) tenemos la capa de embedding del decoder, que se encarga de asociar índices de palabras sobre el diccionario a vectores de longitud ``hidden_size``, que son el elemento con el que opera el mecanismo de atención de la clase ``BahdanauAttention``.

De hecho, en la línea 7 se define el atributo ``attention`` apoyándose en esta clase que se acaba de mencionar. Esta capa de atención dinámica (que depende secuencialmente del token que se elige como query, es decir, del punto de decodificación del input en el que nos encontremos) se encarga de *asociar* entre sí los estados ocultos para modular tanto la longitud como el orden de las palabras en la traducción de una sentencia de input. Esta influencia viene dada por la definición de los estados ocultos del decoder en función del contexto dinámico como 

$$s_i = gru(s_{i-1}, c_i)$$

y esto se lleva a cabo concretamente en el cuerpo de ``forward_step`` donde se producen los outputs del decoder en función de la salida anterior del decoder (línea 42) y el contexto (línea 45).

El atributo ``gru`` (línea 7) modela una GRU que toma como entrada dos vectores de embedding (uno será un estado oculto del decoder y otro el vector de contexto correspondiente) y devuelve un vector que será el próximo estado oculto asociado a la secuencia de outputs del decoder. En la línea 8 se define una capa de transformación lineal para los estados ocultos del decoder. Se encarga de hacer colapsar todo el embedding en una secuencia de tamaño ``output_size`` que en nuestro caso se corresponderá con el tamaño del diccionario del lenguage correspondiente. Esta secuencia resultante se pasará por un softmax para generar las probabilidades de aparición de cada palabra del diccionario como siguiente token de traducción en la salida final.

El último atributo de la clase es ``dropout``, que consiste en un módulo de dropout para el decoder que se aplica solo en el entrenamiento.

<br>

A continuación comentamos los dos métodos de la clase: ``forward`` y ``forward_step``.

El método forward se encarga de realizar el *forward pass* del decoder al completo, para un cierto *batch* de embeddings proporcionado por el encoder. Para esto, se apoya en el método ``forward_step`` que analizaremos en último lugar. El método ``forward`` toma como parámetros de entrada un batch de outputs del encoder (``encoder_outputs``), el último estado oculto asociados a cada vector del batch de outputs del encoder (``encoder_hidden``), y un tensor objetivo ``target_tensor`` para realizar *teacher forcing* sobre el decoder. Por defecto se escoger ``target_tensor = None`` así que no se realiza teacher forcing salvo que se indique explícitamente.

En el cuerpo de ``forward`` se comienza obteniendo el ``batch_size`` a partir del número de vectores en ``encoder_outputs``. Acto seguido, se inicializa el ``decoder_input`` con el token de inicio de sentencia $\text{<SOS>}$ (secuencia de un único token) y el primer estado oculto del decoder (``decoder_hidden``) como el último estado oculto del encoder ``encoder_hidden``. También se crean listas vacías para almacenar secuencialmente los outputs del decoder (``decoder_outputs``) y la atención en cada caso.

Con esto, ya se puede comenzar el bucle de decodificación (líneas 19-32). Se van recorriendo las posiciones de la secuencia hasta completar el máximo de token por secuencia prefijado (``MAX_LENGTH``), y para cada token, se ejecuta ``forward_step`` usando como parámetros de entrada ``decoder_input``, ``decoder_hidden`` y ``encoder_outputs`` para recibir como salida el embedding de output del decoder, el estado oculto que se genera en esa decodificación y los pesos de atención asociados al punto $i$ de la secuencia. Lo siguiente es actualizar las secuencias de ``decoder_outputs`` y ``attentions`` y realizar el teacher forcing (solo si aplica) o dejar como próximo input al decoder la salida del decoder en este paso $i$. 

Al finalizar el bucle se unen todos los ``decoder_outputs`` en un solo tensor de salida de dimensiones [``batch_size``,``MAX_LENGTH``,``output_size``] que respeta la secuancialidad en la lista, y se le pasa un logsoftmax como último paso pervio a generar la traduccion resultante de cada  sentencia en el batch. Finalmente, se devuelven los outputs del decoder concatenados, el último estado oculto del decoder, y las atenciones concatenadas; para cada elemento del batch.

Centrándonos ahora en la llamada a ``forward_step``: La función ejecuta primeramente un dropout sobre los embeddings del input al decoder y calcula la atención entre el estado oculto anterior del decoder y los outputs del encoder (línea 45). A continuación se concatena el input al decoder con el vector de contexto que genera la capa de atención y se pasa a una GRU para que genere un vector de output que se transforma en la capa de salida para ajustarse al tamaño del diccionario. Finalmente se devuelven el output ajustado, el nuevo estado oculto generado por la GRU y los pesos de la atención, que serán los protagonistas en el siguiente análisis.

<br>
<br>

**REFERENCIANDO EL MECANISMO DE VISUALIZACIÓN PARA LA ATENCIÓN**

Para terminar vamos a explicar el funcionamiento del mecanismo que se ha empleado para visualizar los alineamientos entre token de entrada al encoder y tokens de salida del decoder en forma de mapa de calor.

Para poder asociar la descripción del mecanismo a ejemplos concretos, contamos con 4 ejemplos de traducciones que pueden considerarse correctas (o casi) que hemos conseguido tras entrenar en otro fichero el modelo que se explicaba anteriormente.

![Ejemplo 1](../imgs/attn_matrix_example_1.png)
![Ejemplo 2](../imgs/attn_matrix_example_2.png)

$\hspace{4 cm}$ *Ejemplo 1* $\hspace{8 cm}$ *Ejemplo 2*

![Ejemplo 3](../imgs/attn_matrix_example_3.png)
![Ejemplo 4](../imgs/attn_matrix_example_4.png)

$\hspace{4 cm}$ *Ejemplo 3* $\hspace{8 cm}$ *Ejemplo 4*



En el eje $X$ podemos encontrar que cada coordenada horizontal de los píxeles del "mallado coloreado" está asociada a una de las palabras que conforman cierta sentencia de input para el modelo. En este caso, el input es una frase en francés que se lee de izquierda a derecha en el plot, y aparece segmentada en las diferentes palabras que la componen de forma ordenada. Además, se incluye el token reservado $ \hspace{0.1 cm}\text{<EOS>} \hspace{0.1 cm}$ para indicar el final de una oración.

En el eje $Y$ encontramos la misma descomposición ordenada en palabras, pero esta vez de la frase traducida que el modelo da como output al introducir el input del eje $X$. El mallado coloreado que conforman las entradas asociadas a cada par $(x,y)$ es un *heatmap* cuya escala de colores se muestra en la barra de la derecha. Lo que mide este heatmap en cada una de sus entradas $(i,j)$ es el valor del peso de atención $\alpha_{i,j}$ que es devuelto por la capa de atención del decoder para cada token generado como output. Recordando cómo se definía el vector de contexto en cada instante $i$
$$c_i = \sum_{j=1}^{T_x} \alpha_{i,j}h_j$$
notamos que el significado de cada valor $\alpha_{i,j}$ es la ponderación que tendrá la anotación $h_j$ a la hora de generar el contexto asociado a la $i$-ésima palabra del input. En otras palabras, este gráfico de heatmap permite identificar a qué palabras del input presta más atención el modelo de cara a generar cada palabra del output. Es una forma de visualizar el dinamismo del contexto dependiendo de la palabra del output en la que nos encontremos.

Según la definición de $\alpha_{i,j}$, los valores que toma este parámetro están restringidos al intervalo $[0,1]$ y cuanto mayor sea el valor, indica que más atencion recibe la palabra $x_i$ par parte del vector de contexto, al modelar la parte del output asociada a la posición $j$.

Además, esta lectura del gráfico nos permite intuir cómo está funcionando el método de ordenamiento en cada traducción: si nos fijamos en las coordenadas $i$ (eje $Y$) donde $\alpha_{i,j}$ toma mayor valor (se acerca más hacia el blanco en la escala) entendemos que la atención sobre esa palabra inicial $x_i$ influye considerablemente en el output $y_j$ tanto en el significado de esta palabra como en su posición $j$ en la frase de salida.

Por lo general, vemos que el mecanismo de atención presenta un comportamiento aproximadamente diagonal a partir de la segunda palabra traducida. Esto es porque no hay grandes variaciones sintácticas entre el francés y el inglés entre los pares input-traducción que se muestran justamente en estas gráficas. Un ejemplo donde se invierte el orden de palabras es el ejemplo 3, donde "question, idiote" y "stupid, question" evidencian estas diferencias ocasionales en cuanto al orden en la sintaxis. Otra caracterísitca que merece la pena señalar es que en las dos o tres primeras palabras del output generado, la atención se suele trasladar a partes intermedias o final de la frase de entrada. Aunque esto sucede por los procesos internos de optimización del método, podemos dar una intuición que soporte esta "decisión del método", esto es, que la manera de iniciar una frase a la hora de hablar, muchas veces se ve influida por lo que queremos decir después y no es el inicio de la frase el que "se determina a sí mismo".